# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-05-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-04-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-05 05:27:36,33.93911,67.709953,46837,1846,37260,7731.0,Afghanistan,120.315942,3.941328
1,NaN,NaN,NaN,Albania,2020-12-05 05:27:36,41.15330,20.168300,41302,870,20974,19458.0,Albania,1435.193551,2.106436
2,NaN,NaN,NaN,Algeria,2020-12-05 05:27:36,28.03390,1.659600,86730,2492,56079,28159.0,Algeria,197.783209,2.873285
3,NaN,NaN,NaN,Andorra,2020-12-05 05:27:36,42.50630,1.521800,6955,77,6130,748.0,Andorra,9001.488384,1.107117
4,NaN,NaN,NaN,Angola,2020-12-05 05:27:36,-11.20270,17.873900,15493,353,8299,6841.0,Angola,47.139517,2.278448


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,45384,45600,45723,45844,46116,46274,46516,46718,46837,46837
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,34944,35600,36245,36790,37625,38182,39014,39719,40501,41302
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,78025,79110,80168,81212,82221,83199,84152,85084,85927,86730


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1728,1737,1740,1752,1774,1795,1822,1841,1846,1846
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,743,753,771,787,798,810,822,839,852,870
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2329,2352,2372,2393,2410,2431,2447,2464,2480,2492


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36145,36232,36295,36709,36716,36831,36946,37218,37260,37260
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,17031,17352,17755,18152,18481,18849,19384,19912,20484,20974
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,50712,51334,51946,52568,53204,53809,54405,54990,55538,56079


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2686,2704,2716,2735,2751,2780,2818,2873,2893,2945
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,8473,8576,8603,8733,8820,8890,9051,9163,9341,9501
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1170,1170,1171,1173,1175,1178,1189,1206,1214,1217


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,41,42,42,42,42,42,42,42,42,42
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,98,98,98,98,98,98,137,137,137,138
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,10,10,10,10,11,11,11,29,29,29


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
648,1001.0,Autauga,Alabama,US,2020-12-05 05:27:36,32.539527,-86.644082,2945,42,0,2903.0,"Autauga, Alabama, US",5271.259554,1.426146
685,1075.0,Lamar,Alabama,US,2020-12-05 05:27:36,33.779950,-88.096680,734,8,0,726.0,"Lamar, Alabama, US",5316.914162,1.089918
686,1077.0,Lauderdale,Alabama,US,2020-12-05 05:27:36,34.901719,-87.656247,4522,57,0,4465.0,"Lauderdale, Alabama, US",4876.575828,1.260504


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,14367462,278932,5470389
India,9608211,139700,9058822
Brazil,6533968,175964,5825745


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,14367462,278932,5470389,8618141,2020-12-05 05:27:36,37.936303,-91.379001
India,9608211,139700,9058822,409689,2020-12-05 05:27:36,23.088275,81.806127
Brazil,6533968,175964,5825745,532259,2020-12-05 05:27:36,-12.669522,-48.480493
Russia,2382012,41730,1872214,468068,2020-12-05 05:27:36,54.546312,62.120860
France,2321703,54859,173986,2092858,2020-12-05 05:27:36,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,1310307,19734,0
Texas,1299469,22825,0
Florida,1039207,18994,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,1310307,19734,0,1290573,2020-12-05 05:27:36,37.843962,-120.728594
Texas,1299469,22825,0,1276644,2020-12-05 05:27:36,31.660643,-98.653069
Florida,1039207,18994,0,1020213,2020-12-05 05:27:36,28.940755,-82.700744
Illinois,770088,13782,0,756306,2020-12-05 05:27:36,39.843618,-89.178330
New York,685364,34830,0,650534,2020-12-05 05:27:36,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,430583,7842,0
Illinois,Cook,322122,6852,0
Florida,Miami-Dade,238813,3888,0
Arizona,Maricopa,222218,4119,0
Texas,Harris,195558,3046,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,430583,7842,0,422741,2020-12-05 05:27:36,34.308284,-118.228241,6037.0
Illinois,Cook,322122,6852,0,315270,2020-12-05 05:27:36,41.841448,-87.816588,17031.0
Florida,Miami-Dade,238813,3888,0,234925,2020-12-05 05:27:36,25.611236,-80.551706,12086.0
Arizona,Maricopa,222218,4119,0,218099,2020-12-05 05:27:36,33.348359,-112.491815,4013.0
Texas,Harris,195558,3046,0,192512,2020-12-05 05:27:36,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,46718,39719,85084,6842,15319,144,1440103,137231,27939,289461,...,6225,73335,1,103067,1358,90192,10,2217,17700,10129
2020-12-03,46837,40501,85927,6904,15361,144,1447732,138508,27949,293430,...,6455,73431,1,103548,1361,92708,10,2239,17730,10424
2020-12-04,46837,41302,86730,6955,15493,144,1454631,139692,27956,297245,...,6731,73751,1,103877,1361,94676,10,2267,17857,10547


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,1841,839,2464,76,351,4,39156,2228,908,3446,...,80,611,0,905,35,763,1,621,357,277
2020-12-03,1846,852,2480,77,352,4,39305,2254,908,3538,...,80,611,0,909,35,780,1,624,357,280
2020-12-04,1846,870,2492,77,353,4,39512,2277,908,3651,...,80,611,0,913,35,797,1,627,364,281


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,37218,19912,54990,5988,8139,133,1268358,111795,25614,233657,...,4584,70534,0,98030,1201,66989,8,1518,17086,8643
2020-12-03,37260,20484,55538,6066,8244,133,1274675,113228,25629,238127,...,4707,70712,1,98521,1209,68250,8,1525,17102,8754
2020-12-04,37260,20974,56079,6130,8299,133,1281955,114990,25634,243775,...,4826,71002,1,98879,1220,70206,8,1534,17145,8802


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2686,2704,2716,2735,2751,2780,2818,2873,2893,2945
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,650,667,671,679,683,689,701,713,725,734
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,3936,3967,4004,4078,4133,4223,4267,4360,4433,4522
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1248,1263,1273,1279,1296,1316,1334,1353,1373,1399
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7644,7718,7735,7757,7785,7814,7872,7925,8015,8071


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-02,256828,33802,0,340979,161521,1265182,241172,121426,36698,49,...,384285,1268946,202220,4461,1589,242480,170342,49905,420930,34491
2020-12-03,260359,34563,0,346421,164310,1286550,247209,126177,37456,49,...,388252,1286369,206165,4642,1589,244503,172437,51025,426534,35046
2020-12-04,264199,35325,0,352101,167137,1310307,252222,127715,38398,49,...,392608,1299469,209170,4763,1613,247380,174290,52172,432207,35677


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-02        2873    9163    1206  1239   3100     709   1188    6695   
2020-12-03        2893    9341    1214  1252   3158     709   1200    6809   
2020-12-04        2945    9501    1217  1270   3231     711   1211    6939   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-02         1859     1055  ...    1363    341     2075      458   
2020-12-03         1875     1067  ...    1403    344     2094      479   
2020-12-04         1891     1082  ...    1426    352     2134      489   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-02           1934  1678  1121          0      487    415  
2020-12-03           2024  1693  1149          0      497    419  
2020-12-04           2077  1724  1175          0      518    419  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-02,3711,122,0,6739,2522,19410,3193,5091,779,0,...,4688,22335,906,74,23,4113,2850,778,3703,239
2020-12-03,3776,130,0,6821,2555,19553,3320,5111,779,0,...,4781,22573,917,75,23,4147,2900,789,3773,266
2020-12-04,3831,142,0,6885,2586,19734,3338,5146,782,0,...,4876,22825,925,77,23,4160,2925,799,3842,266


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-02          42     137      29   38     43      19     42     121   
2020-12-03          42     137      29   38     45      19     42     121   
2020-12-04          42     138      29   38     46      20     42     126   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-02           50       24  ...       6      8       10        1   
2020-12-03           51       24  ...       6      8       11        2   
2020-12-04           54       24  ...       6      8       11        2   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-02              6     2     4          9        8      2  
2020-12-03             10     2     5          9        8      2  
2020-12-04             10     2     5          9        8      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,0.004343,0.018070,0.011075,0.007658,0.004459,0.014085,0.005258,0.009296,0.000573,0.013913,...,0.033367,0.000805,0.0,0.004346,0.005181,0.024863,0.0,0.009103,0.001981,0.000000
2020-12-03,0.002547,0.019688,0.009908,0.009062,0.002742,0.000000,0.005298,0.009305,0.000358,0.013712,...,0.036948,0.001309,0.0,0.004667,0.002209,0.027896,0.0,0.009923,0.001695,0.029124
2020-12-04,0.000000,0.019777,0.009345,0.007387,0.008593,0.000000,0.004765,0.008548,0.000250,0.013001,...,0.042758,0.004358,0.0,0.003177,0.000000,0.021228,0.0,0.012506,0.007163,0.011800


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,0.010428,0.020681,0.006947,0.000000,0.002857,0.0,0.005857,0.015960,0.0,0.036391,...,0.025641,0.0,NaN,0.00444,0.0,0.021419,0.0,0.003231,0.000000,0.000000
2020-12-03,0.002716,0.015495,0.006494,0.013158,0.002849,0.0,0.003805,0.011670,0.0,0.026698,...,0.000000,0.0,NaN,0.00442,0.0,0.022280,0.0,0.004831,0.000000,0.010830
2020-12-04,0.000000,0.021127,0.004839,0.000000,0.002841,0.0,0.005267,0.010204,0.0,0.031939,...,0.000000,0.0,NaN,0.00440,0.0,0.021795,0.0,0.004808,0.019608,0.003571


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,0.007362,0.027239,0.010753,0.008081,0.026097,0.023077,0.004043,0.012957,0.000312,0.027077,...,0.022530,0.000000,NaN,0.004797,0.005021,0.015939,0.0,0.002642,0.001583,0.000000
2020-12-03,0.001128,0.028726,0.009965,0.013026,0.012901,0.000000,0.004980,0.012818,0.000586,0.019131,...,0.026832,0.002524,inf,0.005009,0.006661,0.018824,0.0,0.004611,0.000936,0.012843
2020-12-04,0.000000,0.023921,0.009741,0.010551,0.006672,0.000000,0.005711,0.015562,0.000195,0.023718,...,0.025281,0.004101,0.0,0.003634,0.009098,0.028659,0.0,0.005902,0.002514,0.005483


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-02,0.015532,0.020746,NaN,0.011390,0.013885,0.015361,0.016274,0.022500,0.009768,0.0,...,0.010782,0.014399,0.020200,0.052371,0.025161,0.010068,0.018694,0.022266,0.010964,0.020293
2020-12-03,0.013749,0.022513,NaN,0.015960,0.017267,0.016889,0.025032,0.039127,0.020655,0.0,...,0.010323,0.013730,0.019508,0.040574,0.000000,0.008343,0.012299,0.022443,0.013313,0.016091
2020-12-04,0.014749,0.022047,NaN,0.016396,0.017205,0.018466,0.020278,0.012189,0.025150,0.0,...,0.011220,0.010184,0.014576,0.026066,0.015104,0.011767,0.010746,0.022479,0.013300,0.018005


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-02      0.019517  0.012374  0.014298  0.029070  0.012741  0.011412   
2020-12-03      0.006961  0.019426  0.006633  0.010492  0.018710  0.000000   
2020-12-04      0.017974  0.017129  0.002471  0.014377  0.023116  0.002821   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-02      0.001686  0.014701  0.017515  0.003806  ...  0.011878   
2020-12-03      0.010101  0.017028  0.008607  0.011374  ...  0.029347   
2020-12-04      0.009167  0.019092  0.008533  0.014058  ...  0.016393   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-02      0.011869  0.024186  0.024609   0.016290  0.013897  0.029385   
2020-12-03      0.008798  0.009157  0.045852   0.046536  0.008939  0.024978   
2020-12-04      0.023256  0.019102  0.020877   0.026186  0.018311  0.022628   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-02            NaN  0.040598  0.014670  
2020-12-03            NaN  0.020534  0.009639  
2020-12-04            NaN  0.042254  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-02,0.020066,0.000000,NaN,0.007776,0.003981,0.005491,0.027018,0.010119,0.002574,NaN,...,0.010781,0.009994,0.017978,0.027778,0.0,0.004886,0.016043,0.026385,0.025478,0.000000
2020-12-03,0.017515,0.065574,NaN,0.012168,0.013085,0.007367,0.039775,0.003929,0.000000,NaN,...,0.019838,0.010656,0.012141,0.013514,0.0,0.008266,0.017544,0.014139,0.018904,0.112971
2020-12-04,0.014566,0.092308,NaN,0.009383,0.012133,0.009257,0.005422,0.006848,0.003851,NaN,...,0.019870,0.011164,0.008724,0.026667,0.0,0.003135,0.008621,0.012674,0.018288,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-02         0.0  0.000000  1.636364  1.111111  0.075000  0.000000   
2020-12-03         0.0  0.000000  0.000000  0.000000  0.046512  0.000000   
2020-12-04         0.0  0.007299  0.000000  0.000000  0.022222  0.052632   

Province_State                                      ... Wyoming         \
Admin2         Butler   Calhoun  Chambers Cherokee  ...    Park Platte   
2020-12-02        0.0  0.000000  0.000000      0.0  ...     0.0    0.0   
2020-12-03        0.0  0.000000  0.020000      0.0  ...     0.0    0.0   
2020-12-04        0.0  0.041322  0.058824      0.0  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2020-12-02          0.0      0.0   0.000000   0.0  0.00        0.0      0.0   
2020-12-03          0.1      1.0   0.666667   0.0  0.25        0.0      0.0   
2020-12-04          0.0      0.0   0.000000   0.0  0.00        0.0      0.0   

Province_State         
Admin2         Weston  
2020-12-02        0.0  
2020-12-03        0.0  
2020-12-04        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,0.004476,0.018812,0.011498,0.007465,0.004805,0.008900,0.005100,0.007741,0.000472,0.012942,...,0.031392,0.001694,0.0,0.003510,0.004024,0.025000,4.751582e-51,0.006866,0.001668,0.006796
2020-12-03,0.003512,0.019250,0.010703,0.008263,0.003773,0.004450,0.005199,0.008523,0.000415,0.013327,...,0.034170,0.001502,0.0,0.004089,0.003117,0.026448,2.375791e-51,0.008395,0.001681,0.017960
2020-12-04,0.001756,0.019514,0.010024,0.007825,0.006183,0.002225,0.004982,0.008536,0.000333,0.013164,...,0.038464,0.002930,0.0,0.003633,0.001558,0.023838,1.187895e-51,0.010450,0.004422,0.014880


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,0.011580,0.018062,0.007202,5.086483e-08,0.004145,6.357829e-07,0.005587,0.014095,0.000069,0.036919,...,0.019172,0.000901,NaN,0.004171,1.606309e-29,0.020787,0.0,0.002353,1.663972e-06,0.001450
2020-12-03,0.007148,0.016778,0.006848,6.578973e-03,0.003497,3.178914e-07,0.004696,0.012882,0.000034,0.031808,...,0.009586,0.000450,NaN,0.004295,8.031544e-30,0.021534,0.0,0.003592,8.319859e-07,0.006140
2020-12-04,0.003574,0.018953,0.005843,3.289486e-03,0.003169,1.589457e-07,0.004981,0.011543,0.000017,0.031874,...,0.004793,0.000225,NaN,0.004348,4.015772e-30,0.021664,0.0,0.004200,9.804338e-03,0.004856


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-12-02,0.005287,0.025734,0.011080,0.009921,0.017660,0.011632,0.004717,0.013755,0.000276,0.026947,...,0.023649,0.001345,NaN,0.004073,0.006297,0.015611,2.784130e-53,0.003047,0.002249,0.005120
2020-12-03,0.003208,0.027230,0.010523,0.011473,0.015280,0.005816,0.004849,0.013287,0.000431,0.023039,...,0.025241,0.001934,NaN,0.004541,0.006479,0.017218,1.392065e-53,0.003829,0.001593,0.008981
2020-12-04,0.001604,0.025576,0.010132,0.011012,0.010976,0.002908,0.005280,0.014424,0.000313,0.023379,...,0.025261,0.003018,0.0,0.004087,0.007789,0.022938,6.960325e-54,0.004865,0.002053,0.007232


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-02,0.013376,0.019083,NaN,0.015339,0.012025,0.013940,0.017399,0.020219,0.013157,9.187413e-80,...,0.013374,0.012004,0.016048,0.034522,0.014173,0.009721,0.016352,0.021293,0.010456,0.019012
2020-12-03,0.013562,0.020798,NaN,0.015650,0.014646,0.015415,0.021216,0.029673,0.016906,4.593707e-80,...,0.011848,0.012867,0.017778,0.037548,0.007086,0.009032,0.014325,0.021868,0.011885,0.017552
2020-12-04,0.014156,0.021422,NaN,0.016023,0.015926,0.016940,0.020747,0.020931,0.021028,2.296853e-80,...,0.011534,0.011525,0.016177,0.031807,0.011095,0.010399,0.012536,0.022174,0.012592,0.017778


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-02      0.015270  0.013034  0.010001  0.017880  0.014968  0.006200   
2020-12-03      0.011116  0.016230  0.008317  0.014186  0.016839  0.003100   
2020-12-04      0.014545  0.016679  0.005394  0.014281  0.019977  0.002961   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-02      0.003912  0.016354  0.016422  0.007432  ...  0.017242   
2020-12-03      0.007006  0.016691  0.012514  0.009403  ...  0.023294   
2020-12-04      0.008086  0.017892  0.010524  0.011731  ...  0.019844   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-02      0.013279  0.021858  0.022204   0.018551  0.014392  0.023119   
2020-12-03      0.011038  0.015507  0.034028   0.032543  0.011665  0.024048   
2020-12-04      0.017147  0.017305  0.027452   0.029365  0.014988  0.023338   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-02           -1.0  0.042801  0.015714  
2020-12-03           -1.0  0.031667  0.012676  
2020-12-04           -1.0  0.036960  0.006338  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-02,0.014515,0.003048,NaN,0.006179,0.005415,0.004870,0.021686,0.007703,0.003891,NaN,...,0.009198,0.008255,0.016203,0.029261,1.290145e-10,0.004763,0.014154,0.023598,0.022474,0.028312
2020-12-03,0.016015,0.034311,NaN,0.009173,0.009250,0.006119,0.030730,0.005816,0.001946,NaN,...,0.014518,0.009455,0.014172,0.021387,6.450724e-11,0.006515,0.015849,0.018868,0.020689,0.070642
2020-12-04,0.015291,0.063309,NaN,0.009278,0.010692,0.007688,0.018076,0.006332,0.002898,NaN,...,0.017194,0.010310,0.011448,0.024027,3.225362e-11,0.004825,0.012235,0.015771,0.019488,0.035321


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-12-02      0.000395  0.100141  0.824435  0.570250  0.038034   
2020-12-03      0.000197  0.050070  0.412218  0.285125  0.042273   
2020-12-04      0.000099  0.028685  0.206109  0.142562  0.032247   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-12-02      2.025779e-08  0.001524  0.001411  0.000241  0.000033  ...   
2020-12-03      1.012889e-08  0.000762  0.000705  0.010120  0.000016  ...   
2020-12-04      2.631579e-02  0.000381  0.021014  0.034472  0.000008  ...   

Province_State   Wyoming                                          \
Admin2              Park    Platte  Sheridan Sublette Sweetwater   
2020-12-02      0.000489  0.036561  0.000009     0.00   0.000031   
2020-12-03      0.000244  0.018281  0.050004     0.50   0.333349   
2020-12-04      0.000122  0.009140  0.025002     0.25   0.166674   

Province_State                                                                
Admin2                 Teton         Uinta    Unassigned  Washakie    Weston  
2020-12-02      9.313226e-10  9.934335e-09 -6.146820e-08  0.000558  0.250489  
2020-12-03      4.656613e-10  1.250000e-01 -2.048940e-08  0.000279  0.125122  
2020-12-04      2.328306e-10  6.250000e-02 -8.781171e-09  0.000140  0.062531  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201205,AK,34963,NaN,1032268,NaN,totalTestsViral,1067231,166.0,799.0,...,1067231,1,5,d8b27967afb185d8e4f38f5209d713ef23065d88,0,0,0,0,0,NaN
1,20201205,AL,267589,45785.0,1415357,NaN,totalTestsPeopleViral,1637161,1867.0,26331.0,...,1682946,46,0,7a521812788c45e903b5a08f357bdd78f668340a,0,0,0,0,0,NaN
2,20201205,AR,169382,22671.0,1601735,NaN,totalTestsViral,1748446,1056.0,9380.0,...,1771117,34,51,1570d9633ddb7c4522881fa225446b3967f35981,0,0,0,0,0,NaN
3,20201205,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,8fc9cfe6620fead382a0d7ce38f843c84469c5e1,0,0,0,0,0,NaN
4,20201205,AZ,358900,12139.0,2003152,NaN,totalTestsPeopleViral,2349913,2931.0,28006.0,...,2362052,40,268,b22ef8b238dd16e3fba0bcc29553952e5b2414a4,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201205,AK,34963.0,NaN,1032268.0,NaN,totalTestsViral,1067231.0,166.0,799.0,...,1067231,1,5,d8b27967afb185d8e4f38f5209d713ef23065d88,0,0,0,0,0,NaN
1,20201205,AL,267589.0,45785.0,1415357.0,NaN,totalTestsPeopleViral,1637161.0,1867.0,26331.0,...,1682946,46,0,7a521812788c45e903b5a08f357bdd78f668340a,0,0,0,0,0,NaN
2,20201205,AR,169382.0,22671.0,1601735.0,NaN,totalTestsViral,1748446.0,1056.0,9380.0,...,1771117,34,51,1570d9633ddb7c4522881fa225446b3967f35981,0,0,0,0,0,NaN
3,20201205,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,8fc9cfe6620fead382a0d7ce38f843c84469c5e1,0,0,0,0,0,NaN
4,20201205,AZ,358900.0,12139.0,2003152.0,NaN,totalTestsPeopleViral,2349913.0,2931.0,28006.0,...,2362052,40,268,b22ef8b238dd16e3fba0bcc29553952e5b2414a4,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-05,AK,34963,NaN,1032268,NaN,totalTestsViral,1067231,166.0,799.0,NaN,...,1067231,1,5,d8b27967afb185d8e4f38f5209d713ef23065d88,0,0,0,0,0,NaN
2020-12-05,AL,267589,45785.0,1415357,NaN,totalTestsPeopleViral,1637161,1867.0,26331.0,NaN,...,1682946,46,0,7a521812788c45e903b5a08f357bdd78f668340a,0,0,0,0,0,NaN
2020-12-05,AR,169382,22671.0,1601735,NaN,totalTestsViral,1748446,1056.0,9380.0,359.0,...,1771117,34,51,1570d9633ddb7c4522881fa225446b3967f35981,0,0,0,0,0,NaN
2020-12-05,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,8fc9cfe6620fead382a0d7ce38f843c84469c5e1,0,0,0,0,0,NaN
2020-12-05,AZ,358900,12139.0,2003152,NaN,totalTestsPeopleViral,2349913,2931.0,28006.0,701.0,...,2362052,40,268,b22ef8b238dd16e3fba0bcc29553952e5b2414a4,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-05,AK,34963.0,NaN,1032268.0,NaN,totalTestsViral,1067231.0,166.0,799.0,NaN,...,1067231,1,5,d8b27967afb185d8e4f38f5209d713ef23065d88,0,0,0,0,0,NaN
2020-12-05,AL,267589.0,45785.0,1415357.0,NaN,totalTestsPeopleViral,1637161.0,1867.0,26331.0,NaN,...,1682946,46,0,7a521812788c45e903b5a08f357bdd78f668340a,0,0,0,0,0,NaN
2020-12-05,AR,169382.0,22671.0,1601735.0,NaN,totalTestsViral,1748446.0,1056.0,9380.0,359.0,...,1771117,34,51,1570d9633ddb7c4522881fa225446b3967f35981,0,0,0,0,0,NaN
2020-12-05,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,8fc9cfe6620fead382a0d7ce38f843c84469c5e1,0,0,0,0,0,NaN
2020-12-05,AZ,358900.0,12139.0,2003152.0,NaN,totalTestsPeopleViral,2349913.0,2931.0,28006.0,701.0,...,2362052,40,268,b22ef8b238dd16e3fba0bcc29553952e5b2414a4,0,0,0,0,0,NaN
